# Primeiro dataset

Dataset escolhido: [https://archive.ics.uci.edu/dataset/1/abalone](https://archive.ics.uci.edu/dataset/1/abalone)

## Extraindo os dados

In [39]:
import pandas as pd

path_to_data = 'data/abalone.data'

# Carregar base de dados
# DataFrame
dataset = pd.read_csv(path_to_data, header=None)

## Transformando os dados

In [40]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

# Obter a coluna 0 e aplicar o OneHotEncoder (coluna 'Sexo')
SEX_COLUMN = 0
column_transformer = make_column_transformer((OneHotEncoder(), [SEX_COLUMN]), remainder='passthrough')

dataset = column_transformer.fit_transform(dataset)

dataset = pd.DataFrame(data=dataset)

## Separando os dados
Seperação dos dados em features (X) e target (y)

In [41]:
columns = len(dataset.columns)

y = dataset[columns-1] # extrai o target (última coluna)
X = dataset.loc[:,0:columns-2]

## Normalizando os dados
Deixando os valores de todas as colunas entre 0 e 1

In [42]:
from sklearn.preprocessing import MinMaxScaler

x_values = X.values
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x_values)
X = pd.DataFrame(x_scaled)

## Criando os folds
Serão 10 folds para o k-fold cross validation

In [43]:
import numpy as np

# Transforma para array NumPy
X = np.array(X)
y = np.array(y)

# Define o número de folds
N_FOLDS = 10

## Obtendo os scores dos modelos

### Árvore de decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

entropy_tree_model = DecisionTreeClassifier(criterion="entropy")
# TODO - obter também o erro médio
# TODO - analisar o warning de classe com apenas 1 membro
entropy_tree_scores = cross_val_score(entropy_tree_model, X, y, cv=N_FOLDS)

print(entropy_tree_scores)

print(f"Média de acerto da ávore de decisão com entropy: {round(np.mean(entropy_tree_scores) * 100)}%")

/home/yago/Development/topicos_especiais/mlalgos/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


[0.12679426 0.17703349 0.18899522 0.24641148 0.19138756 0.19617225
 0.21052632 0.18944844 0.19184652 0.23261391]
Média de acerto da ávore de decisão com entropy: 20%
